# Thread level Persistence

#### **1. Checkpointers (in LangGraph)**

* Think of a **checkpointer** like a “save game” in a video game.
* It **remembers the state** of your workflow (like what data you’ve processed, what step you’re on).
* If something crashes or you want to resume later, the checkpointer lets you **continue from where you left off** instead of starting from scratch.

Example:
You’re processing 100 documents. After 50, your program stops.

* With a checkpointer → next time, it resumes at document 51.
* Without a checkpointer → it starts again from document 1.

---

#### **2. Threads (in LangGraph)**

* A **thread** is like a “conversation line” or “execution flow.”
* Each user session or workflow run can have its own **thread ID**.
* This keeps data and context separate between different tasks or users.

Example:

* User A’s chatbot session = Thread 1
* User B’s chatbot session = Thread 2
* They don’t mix, because each thread has its own memory and checkpoints.

---

#### **3. State Snapshot**

* A **snapshot** is like taking a photo of your program’s current state.
* It freezes everything (inputs, outputs, progress) so you can **restore it later**.
* Snapshots are created by checkpointers to know exactly where to restart.

Example: Chatbot saves a snapshot after every reply. If it crashes, it reloads the last snapshot and continues.

---

#### **4. Short-term Memory**

* Short-term memory = what the system remembers **within a single thread/session**.
* It’s like remembering the last few messages, but it doesn’t survive forever.

Example: The chatbot remembers what you said 2 minutes ago in the same session.

---

### **5. Long-term Memory**

* Long-term memory = what’s saved **permanently across sessions/threads**.
* It survives restarts and can be recalled even in a new conversation.

Example: The chatbot remembers your name and preferences even if you come back tomorrow.

---



# Short-term memory with the InMemorySaver

In [13]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage, RemoveMessage
from langgraph.checkpoint.memory import InMemorySaver
import config

In [2]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    google_api_key=config.gemini_api_key,
    temperature=0,
    max_tokens=250,
    model_kwargs={"seed": 42}
)

In [3]:
def AskGeminiAI(question):
    response = llm.invoke(question)
    return response

In [4]:
class State(MessagesState):
    summary: str

In [5]:
def ask_question(state: State) -> State:
    
    print(f"\n-------> ENTERING ask_question:")
    
    question = "What is your question?"
    print(question)
    
    return State(messages = [AIMessage(question), HumanMessage(input())])

In [6]:
def chatbot(state: State) -> State:
    
    print(f"\n-------> ENTERING chatbot:")
        
    system_message = f'''
    Here's a quick summary of what's been discussed so far:
    {state.get("summary", "")}
    
    Keep this in mind as you answer the next question.
    '''
    
    response = AskGeminiAI([SystemMessage(system_message)] + state["messages"])
    response.pretty_print()
    
    return State(messages = [response])

In [7]:
def summarize_messages(state: State) -> State:
    print(f"\n-------> ENTERING summarize_messages:")

    new_conversation = ""
    for i in state["messages"]:
        new_conversation += f"{i.type}: {i.content}\n\n"
    
    summary_instructions = f'''
        Update the ongoing summary by incorporating the new lines of conversation below. 
        Build upon the previous summary rather than repeating it, 
        so that the result reflects the most recent context and developments.
        Respond only with the summary.
        
        Previous Summary:
        {state.get("summary", "")}
        
        New Conversation:
        {new_conversation}
        '''

    print(summary_instructions)

    summary = AskGeminiAI([HumanMessage(summary_instructions)])
    
    remove_messages = [RemoveMessage(id = i.id) for i in state["messages"][:]]

    return State(messages = remove_messages, summary = summary.content)

In [16]:
graph = StateGraph(State)

graph.add_node("ask_question", ask_question)
graph.add_node("chatbot", chatbot)
graph.add_node("summarize_messages", summarize_messages)

graph.add_edge(START, "ask_question")
graph.add_edge("ask_question", "chatbot")
graph.add_edge("chatbot", "summarize_messages")
graph.add_edge("summarize_messages", END)

In [17]:
checkpointer = InMemorySaver()
graph_compiled = graph.compile(checkpointer)

1. **`InMemorySaver()`**

   * This is your **checkpointer**.
   * It stores the graph’s progress (state, snapshots) **in memory (RAM)**.
   * So if you run the graph again in the same session, it can remember where it left off.
   * But once you stop the program, this memory is gone (not saved to disk).

2. **`graph.compile(checkpointer)`**

   * This takes your `graph` and **adds checkpointing ability** to it.
   * So every time the graph runs, it can **save and reload snapshots** using the checkpointer.

---    

In [18]:
print(graph_compiled.get_graph().draw_ascii())

    +-----------+      
    | __start__ |      
    +-----------+      
           *           
           *           
           *           
   +--------------+    
   | ask_question |    
   +--------------+    
           *           
           *           
           *           
      +---------+      
      | chatbot |      
      +---------+      
           *           
           *           
           *           
+--------------------+ 
| summarize_messages | 
+--------------------+ 
           *           
           *           
           *           
      +---------+      
      | __end__ |      
      +---------+      


In [19]:
config1 = {"configurable": {"thread_id": "1"}}
config2 = {"configurable": {"thread_id": "2"}}

### What this does

1. **`config1` and `config2`**

   * These are **thread configurations**.
   * Each has a different `thread_id` (`"1"` vs `"2"`).
   * That means LangGraph will treat them as **two separate conversation flows**.

2. **`graph_compiled.invoke(State(), config2)`**

   * You’re starting the graph with an initial `State()` in **thread 2**.
   * Because the `thread_id` is `2`, LangGraph will **store and checkpoint progress** separately from thread 1.
   * If you invoked with `config1`, it would continue the progress of **thread 1**, not thread 2.

---

In [20]:
graph_compiled.invoke(State(), config1)


-------> ENTERING ask_question:
What is your question?


 My pet name is Sonu.



-------> ENTERING chatbot:
================================== Ai Message ==================================

Okay, I will remember that your pet name is Sonu.

-------> ENTERING summarize_messages:

        Update the ongoing summary by incorporating the new lines of conversation below. 
        Build upon the previous summary rather than repeating it, 
        so that the result reflects the most recent context and developments.
        Respond only with the summary.

        Previous Summary:
        

        New Conversation:
        ai: What is your question?

human: My pet name is Sonu.

ai: Okay, I will remember that your pet name is Sonu.


        


{'messages': [],
 'summary': 'The AI is interacting with a user and has asked a question. The user responded by stating their pet name is Sonu, and the AI has acknowledged this and stated it will remember the information.'}

In [21]:
graph_compiled.invoke(State(), config1)


-------> ENTERING ask_question:
What is your question?


 What is my pet name?



-------> ENTERING chatbot:
================================== Ai Message ==================================

Your pet name is Sonu.

-------> ENTERING summarize_messages:

        Update the ongoing summary by incorporating the new lines of conversation below. 
        Build upon the previous summary rather than repeating it, 
        so that the result reflects the most recent context and developments.
        Respond only with the summary.

        Previous Summary:
        The AI is interacting with a user and has asked a question. The user responded by stating their pet name is Sonu, and the AI has acknowledged this and stated it will remember the information.

        New Conversation:
        ai: What is your question?

human: What is my pet name?

ai: Your pet name is Sonu.


        


{'messages': [],
 'summary': "The AI is interacting with a user and has asked a question. The user responded by stating their pet name is Sonu, and the AI has acknowledged this and stated it will remember the information. The AI then asked the user what their question was, and the user asked what their pet name was. The AI then correctly stated the user's pet name is Sonu."}

In [22]:
graph_compiled.invoke(State(), config2)


-------> ENTERING ask_question:
What is your question?


 Do you know my pet name?



-------> ENTERING chatbot:
================================== Ai Message ==================================

As a large language model, I have no memory of past conversations and no personal connection to you. Therefore, I do not know your pet's name.

-------> ENTERING summarize_messages:

        Update the ongoing summary by incorporating the new lines of conversation below. 
        Build upon the previous summary rather than repeating it, 
        so that the result reflects the most recent context and developments.
        Respond only with the summary.

        Previous Summary:
        

        New Conversation:
        ai: What is your question?

human: Do you know my pet name?

ai: As a large language model, I have no memory of past conversations and no personal connection to you. Therefore, I do not know your pet's name.


        


{'messages': [],
 'summary': "The AI has stated it has no memory of past conversations and no personal connection to the user. Consequently, it does not know the user's pet's name."}

# The StateSnapshot class

In [23]:
graph_states = [i for i in graph_compiled.get_state_history(config1)]

In [24]:
graph_states

[StateSnapshot(values={'messages': [], 'summary': "The AI is interacting with a user and has asked a question. The user responded by stating their pet name is Sonu, and the AI has acknowledged this and stated it will remember the information. The AI then asked the user what their question was, and the user asked what their pet name was. The AI then correctly stated the user's pet name is Sonu."}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a02d1-4711-6925-8008-be8058a9c95f'}}, metadata={'source': 'loop', 'step': 8, 'parents': {}}, created_at='2025-10-03T07:46:32.541418+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0a02d1-3d3e-61db-8007-5deabd3c7670'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'messages': [AIMessage(content='What is your question?', additional_kwargs={}, response_metadata={}, id='615a29c2-7d00-4a2b-8d8c-dbc899b668a1'), HumanMessage(content='What is my pet name?

In [28]:
for i in graph_states[::-1]:
    print(f'''
            Messages: {i.values["messages"]}
            Summary: {i.values.get("summary", "")}
            Next: {i.next}
            Step: {i.metadata["step"]}
            '''
         )


            Messages: []
            Summary: 
            Next: ('__start__',)
            Step: -1
            

            Messages: []
            Summary: 
            Next: ('ask_question',)
            Step: 0
            

            Messages: [AIMessage(content='What is your question?', additional_kwargs={}, response_metadata={}, id='7da62dc5-afbc-4906-8eda-eb6174b9ec22'), HumanMessage(content='My pet name is Sonu.', additional_kwargs={}, response_metadata={}, id='a301e56e-9169-44b1-8ff2-f584c1c804a0')]
            Summary: 
            Next: ('chatbot',)
            Step: 1
            

            Messages: [AIMessage(content='What is your question?', additional_kwargs={}, response_metadata={}, id='7da62dc5-afbc-4906-8eda-eb6174b9ec22'), HumanMessage(content='My pet name is Sonu.', additional_kwargs={}, response_metadata={}, id='a301e56e-9169-44b1-8ff2-f584c1c804a0'), AIMessage(content='Okay, I will remember that your pet name is Sonu.', additional_kwargs={}, response_me

* `[::-1]` → this is Python’s way of **reversing a list**.

  * `list[start:stop:step]` → and here the step is `-1`, which means "go backwards".

So `graph_states[::-1]` gives you the list of states **in reverse order** (latest one first).

# Long-term memory with SQLite

If you want long-term memory in LangGraph, instead of using an in-memory saver (which disappears when the program ends), you can use a persistent checkpointer like SQLite. That way, the graph’s state and snapshots are stored on disk and can be recalled across sessions.

In [ ]:
db_path = "C:/Users/Suraj S G/Desktop/LangGraph_DB/langgraph.db"
con = sqlite3.connect(database = db_path, check_same_thread = False)

In [ ]:
checkpointer = SqliteSaver(con)
graph_compiled = graph.compile(checkpointer)

In [ ]:
config1 = {"configurable": {"thread_id": "1"}}

In [ ]:
graph_compiled.invoke(State(), config1)